<a href="https://colab.research.google.com/github/sr6awi/ieee_fraud_detection/blob/main/notebooks/03_Preprocessing_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📘 IEEE-CIS Fraud Detection — 03_Preprocessing.ipynb
**Phase:** Data Preprocessing  
**Owner:** *Salem Ihsan Abidrabbu*  
**Repository:** [github.com/sr6awi/ieee_fraud_detection](https://github.com/sr6awi/ieee_fraud_detection)

---

## 🎯 Stage Overview

The **Preprocessing** phase is responsible for preparing the IEEE-CIS Fraud Detection dataset for the modeling stage by ensuring that all data is **clean**, **integrated**, and **transformed** into a consistent, learning-ready format.

This notebook follows a **structured five-stage preprocessing pipeline** aligned with enterprise-level data engineering best practices:

1️⃣ **Data Cleaning** → Handle missing values, correct outliers, and standardize formatting  
2️⃣ **Data Integration** → Merge related tables into unified datasets  
3️⃣ **Data Transformation** → Encode categorical features and scale numeric ones  
4️⃣ **Data Reduction** → Remove redundant or low-variance attributes  
5️⃣ **Data Discretization** → Convert continuous variables into bins when useful  

---

## ⚙️ Current Focus: Data Cleaning
We begin by systematically cleaning the dataset to ensure quality and consistency.  
This includes:
- Handling missing values (removal / imputation)  
- Detecting and correcting outliers  
- Standardizing column names, types, and formatting  

> 🧭 Once this stage is complete, we’ll move forward to **Data Integration**.

## 🧼 1. Data Cleaning
**Objective:**  
Ensure the dataset is accurate, consistent, and reliable before any transformations or integrations.  
This phase focuses on eliminating noise, fixing irregularities, and preparing the data for deeper preprocessing.

---

### 📊 Sub-Steps & Strategy

#### 1.1 Handling Missing Values
- Inspect missingness across all features.  
- Apply thresholds (e.g., drop columns with >80% missing values).  
- Impute remaining values:  
  - Numerical → median or mean  
  - Categorical → mode or “missing” label  
- Document all imputation strategies.

#### 1.2 Correcting Outliers
- Identify outliers using percentile or IQR method.  
- Apply capping or log-transformation on skewed continuous features such as `TransactionAmt`.  
- Visual validation via boxplots (optional).

#### 1.3 Standardizing Formatting
- Normalize column names (lowercase, underscores).  
- Align data types (numeric, categorical, datetime).  
- Convert date/time fields into consistent formats.  
- Remove redundant spaces or symbols in string fields.

---

> 🧭 Once cleaning is complete, we’ll proceed to **Data Integration** (merging transaction and identity datasets).


### 🧩 1.1 Handling Missing Values

**Objective:**  
Detect and handle missing values systematically to ensure data completeness and prevent bias or data leakage during modeling.

---

#### 🔍 Step 1: Inspection
We’ll begin by:
- Calculating the total and percentage of missing values for each feature.  
- Visualizing the overall missingness to identify heavily incomplete features.  
- Documenting all columns exceeding the **80% missing threshold** (to be dropped).

> Rationale: Columns with excessive missingness often contribute little predictive power while increasing noise and model complexity.

---

#### 🧮 Step 2: Threshold-Based Removal
- Drop columns with more than **80% missing values**.  
- Keep the record of removed columns in a list for documentation and reproducibility.

---

#### 🧰 Step 3: Imputation Strategy
For the remaining columns:
- **Numerical features:** Replace missing values with the **median** (robust to outliers).  
- **Categorical features:** Replace with the **mode** or a special label `"missing"`.  
- **Date/time features:** If applicable, forward-fill or replace with a reference timestamp (depending on feature meaning).

---

#### 🧾 Step 4: Verification
- Confirm that no missing values remain (except for columns intentionally retained).  
- Save an updated summary report (`missing_report.csv`) for audit tracking.

> ✅ After handling missing values, we’ll move to **1.2 Correcting Outliers**.


==============================================================
##⚙️ Configuration & Imports
## --------------------------------------------------------------
## Purpose: Prepare environment, imports, and directory structure
==============================================================

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import json
import os


# === Define project paths ===
BASE_DIR = Path("/content/drive/MyDrive/ML_Projects/ieee-fraud-detection")
DATA_DIR = BASE_DIR / "raw"
PROCESSED_DIR = BASE_DIR / "processed"
ARTIFACT_DIR = BASE_DIR / "artifacts/missing_reports"
ARTIFACT_DIR.mkdir(parents=True, exist_ok=True)

# === Global constants ===
THRESHOLD = 0.80   # 80% missing threshold
SEED = 42

# === Display settings ===
pd.set_option("display.max_rows", 200)

pd.set_option("display.max_columns", 200)

print("✅ Config & directories ready.")
print(f"📂 Data path : {DATA_DIR}")
print(f"🧾 Artifacts : {ARTIFACT_DIR}")

✅ Config & directories ready.
📂 Data path : /content/drive/MyDrive/ML_Projects/ieee-fraud-detection/raw
🧾 Artifacts : /content/drive/MyDrive/ML_Projects/ieee-fraud-detection/artifacts/missing_reports


## Define file paths

In [2]:
PATHS = {
    "train_transaction": DATA_DIR / "train_transaction.csv",
    "train_identity":    DATA_DIR / "train_identity.csv",
    "test_transaction":  DATA_DIR / "test_transaction.csv",
    "test_identity":     DATA_DIR / "test_identity.csv",
}

In [5]:
def _assert_exists(p: Path):
    if not p.exists():
        raise FileNotFoundError(f" File not found: {p}")

##Load dataset

In [4]:
datasets = {}
for name, path in PATHS.items():
    _assert_exists(path)
    print(f" Loading {name} ...")
    df = pd.read_csv(path, low_memory=False)
    datasets[name] = df
    print(f"   → shape: {df.shape}")

print("\n All datasets loaded successfully!")

 Loading train_transaction ...
   → shape: (590540, 394)
 Loading train_identity ...
   → shape: (144233, 41)
 Loading test_transaction ...
   → shape: (506691, 393)
 Loading test_identity ...
   → shape: (141907, 41)

 All datasets loaded successfully!


#Missingness Inspection & Report Generation


###  Inspect missing values, flag columns over threshold , and save reports for audit.

In [5]:
def compute_missing_report(df: pd.DataFrame, name: str, threshold: float = THRESHOLD) -> pd.DataFrame:
    """Compute missingness count and percentage for each feature."""
    n = len(df)
    missing_count = df.isna().sum()
    missing_pct = (missing_count / n).round(4)

    report = (
        pd.DataFrame({
            "feature": df.columns,
            "dtype": df.dtypes.astype(str).values,
            "n_missing": missing_count.values,
            "pct_missing": missing_pct.values
        })
        .sort_values("pct_missing", ascending=False)
        .reset_index(drop=True)
    )
    report["over_threshold"] = report["pct_missing"] > threshold
    return report

In [6]:
def save_missing_report(df_report: pd.DataFrame, name: str):
    """Save individual missingness report per dataset."""
    out_path = ARTIFACT_DIR / f"{name}_missing_report.csv"
    df_report.to_csv(out_path, index=False)
    return out_path

In [7]:
columns_to_drop = {}

for name, df in datasets.items():
    print(f"🔍 Analyzing missing values for: {name}")
    report = compute_missing_report(df, name)
    csv_path = save_missing_report(report, name)
    print(f"   → Report saved to: {csv_path}")

    # Record columns exceeding threshold
    over = report.loc[report["over_threshold"], "feature"].tolist()
    columns_to_drop[name] = over

    print(f"   → Columns over {int(THRESHOLD*100)}% missing: {len(over)}")
    display(report.head(10))  # Preview top 10 most missing features

🔍 Analyzing missing values for: train_transaction
   → Report saved to: /content/drive/MyDrive/ML_Projects/ieee-fraud-detection/artifacts/missing_reports/train_transaction_missing_report.csv
   → Columns over 80% missing: 55


,feature,dtype,n_missing,pct_missing,over_threshold
0,dist2,float64,552913,0.9363,True
1,D7,float64,551623,0.9341,True
2,D13,float64,528588,0.8951,True
3,D14,float64,528353,0.8947,True
4,D12,float64,525823,0.8904,True
5,D6,float64,517353,0.8761,True
6,D9,float64,515614,0.8731,True
7,D8,float64,515614,0.8731,True
8,V153,float64,508595,0.8612,True
9,V140,float64,508595,0.8612,True


🔍 Analyzing missing values for: train_identity
   → Report saved to: /content/drive/MyDrive/ML_Projects/ieee-fraud-detection/artifacts/missing_reports/train_identity_missing_report.csv
   → Columns over 80% missing: 9


,feature,dtype,n_missing,pct_missing,over_threshold
0,id_24,float64,139486,0.9671,True
1,id_25,float64,139101,0.9644,True
2,id_07,float64,139078,0.9643,True
3,id_08,float64,139078,0.9643,True
4,id_21,float64,139074,0.9642,True
5,id_22,float64,139064,0.9642,True
6,id_23,object,139064,0.9642,True
7,id_27,object,139064,0.9642,True
8,id_26,float64,139070,0.9642,True
9,id_18,float64,99120,0.6872,False


🔍 Analyzing missing values for: test_transaction
   → Report saved to: /content/drive/MyDrive/ML_Projects/ieee-fraud-detection/artifacts/missing_reports/test_transaction_missing_report.csv
   → Columns over 80% missing: 52


,feature,dtype,n_missing,pct_missing,over_threshold
0,dist2,float64,470255,0.9281,True
1,D7,float64,446558,0.8813,True
2,D12,float64,437437,0.8633,True
3,D8,float64,432353,0.8533,True
4,D9,float64,432353,0.8533,True
5,V140,float64,430906,0.8504,True
6,V155,float64,430906,0.8504,True
7,V153,float64,430906,0.8504,True
8,V154,float64,430906,0.8504,True
9,V149,float64,430906,0.8504,True


🔍 Analyzing missing values for: test_identity
   → Report saved to: /content/drive/MyDrive/ML_Projects/ieee-fraud-detection/artifacts/missing_reports/test_identity_missing_report.csv
   → Columns over 80% missing: 9


,feature,dtype,n_missing,pct_missing,over_threshold
0,id-24,float64,137167,0.9666,True
1,id-25,float64,136868,0.9645,True
2,id-26,float64,136860,0.9644,True
3,id-08,float64,136848,0.9643,True
4,id-23,object,136845,0.9643,True
5,id-22,float64,136845,0.9643,True
6,id-21,float64,136848,0.9643,True
7,id-27,object,136845,0.9643,True
8,id-07,float64,136848,0.9643,True
9,id-18,float64,91032,0.6415,False


In [8]:
# Save summary of columns exceeding threshold
json_path = ARTIFACT_DIR / "columns_over_threshold.json"
with open(json_path, "w") as f:
    json.dump(columns_to_drop, f, indent=2)

print(f"\n Saved threshold summary JSON → {json_path}")


 Saved threshold summary JSON → /content/drive/MyDrive/ML_Projects/ieee-fraud-detection/artifacts/missing_reports/columns_over_threshold.json


In [9]:
# Quick overview table
summary_data = [
    {"dataset": name,
     "total_features": len(df.columns),
     f"features_over_{int(THRESHOLD*100)}pct_missing": len(over)}
    for name, over in columns_to_drop.items()
]
summary_df = pd.DataFrame(summary_data)
display(summary_df)

print("\n Missingness inspection complete — reports generated successfully.")

,dataset,total_features,features_over_80pct_missing
0,train_transaction,41,55
1,train_identity,41,9
2,test_transaction,41,52
3,test_identity,41,9



 Missingness inspection complete — reports generated successfully.


Threshold Based Removal

In [10]:
# verify processed directory exists
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

dropped_summary = {}  # keep record of dropped columns per dataset
cleaned_datasets = {}  # hold resulting DataFrames

for name, df in datasets.items():
    print(f"\n Processing {name} ...")
    before_shape = df.shape

    # get list of columns to drop for this dataset
    drop_cols = columns_to_drop.get(name, [])
    dropped_summary[name] = drop_cols

    # drop and store cleaned version
    df_cleaned = df.drop(columns=drop_cols, errors="ignore")
    cleaned_datasets[name] = df_cleaned
    after_shape = df_cleaned.shape

    print(f"   → Dropped {len(drop_cols)} columns.")
    print(f"   → Shape before: {before_shape} | after: {after_shape}")



 Processing train_transaction ...
   → Dropped 55 columns.
   → Shape before: (590540, 394) | after: (590540, 339)

 Processing train_identity ...
   → Dropped 9 columns.
   → Shape before: (144233, 41) | after: (144233, 32)

 Processing test_transaction ...
   → Dropped 52 columns.
   → Shape before: (506691, 393) | after: (506691, 341)

 Processing test_identity ...
   → Dropped 9 columns.
   → Shape before: (141907, 41) | after: (141907, 32)


In [11]:
import pandas as pd

def shape_diff(before: dict, after: dict) -> pd.DataFrame:
    rows = []
    for name in before.keys():
        b = before[name].shape
        a = after[name].shape
        rows.append({
            "dataset": name,
            "rows": b[0],
            "before_cols": b[1],
            "after_cols": a[1],
            "dropped_cols": b[1] - a[1],
        })
    return pd.DataFrame(rows).sort_values("dataset").reset_index(drop=True)

diff_df = shape_diff(datasets, cleaned_datasets)
display(diff_df)

# Show a few concrete dropped column names per dataset (for audit)
for name in cleaned_datasets.keys():
    dropped = list(set(datasets[name].columns) - set(cleaned_datasets[name].columns))
    dropped_sorted = sorted(dropped)
    print(f"\n{name}: dropped {len(dropped_sorted)} columns")
    print("Examples:", dropped_sorted[:15])  # first 15 as a quick glance

,dataset,rows,before_cols,after_cols,dropped_cols
0,test_identity,141907,41,32,9
1,test_transaction,506691,393,341,52
2,train_identity,144233,41,32,9
3,train_transaction,590540,394,339,55



train_transaction: dropped 55 columns
Examples: ['D12', 'D13', 'D14', 'D6', 'D7', 'D8', 'D9', 'V138', 'V139', 'V140', 'V141', 'V142', 'V143', 'V144', 'V145']

train_identity: dropped 9 columns
Examples: ['id_07', 'id_08', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27']

test_transaction: dropped 52 columns
Examples: ['D12', 'D7', 'D8', 'D9', 'V138', 'V139', 'V140', 'V141', 'V142', 'V143', 'V144', 'V145', 'V146', 'V147', 'V148']

test_identity: dropped 9 columns
Examples: ['id-07', 'id-08', 'id-21', 'id-22', 'id-23', 'id-24', 'id-25', 'id-26', 'id-27']


In [12]:
import os
from pathlib import Path
import pandas as pd

rows = []
for name in cleaned_datasets.keys():
    p = PROCESSED_DIR / f"{name}_cleaned.csv"
    exists = p.exists()
    size_mb = round(os.path.getsize(p) / (1024*1024), 2) if exists else None
    rows.append({"file": p.name, "exists": exists, "size_MB": size_mb})
confirm_df = pd.DataFrame(rows)
display(confirm_df)

# Optional: quick peek at the cleaned train_transaction (small sample)
sample_path = PROCESSED_DIR / "train_transaction_cleaned.csv"
if sample_path.exists():
    display(pd.read_csv(sample_path, nrows=3))

,file,exists,size_MB
0,train_transaction_cleaned.csv,False,None
1,train_identity_cleaned.csv,False,None
2,test_transaction_cleaned.csv,False,None
3,test_identity_cleaned.csv,False,None


In [13]:
from pathlib import Path

print("🔍 BASE_DIR:", BASE_DIR)
print("🔍 PROCESSED_DIR:", PROCESSED_DIR)
print("\nContents of PROCESSED_DIR right now:")
list(PROCESSED_DIR.glob("*"))

🔍 BASE_DIR: /content/drive/MyDrive/ML_Projects/ieee-fraud-detection
🔍 PROCESSED_DIR: /content/drive/MyDrive/ML_Projects/ieee-fraud-detection/processed

Contents of PROCESSED_DIR right now:


[]

In [14]:
print(PROCESSED_DIR)
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)
print("✅ Exists:", PROCESSED_DIR.exists())

/content/drive/MyDrive/ML_Projects/ieee-fraud-detection/processed
✅ Exists: True


In [15]:
for name, df_cleaned in cleaned_datasets.items():
    out_path = PROCESSED_DIR / f"{name}_cleaned.csv"
    df_cleaned.to_csv(out_path, index=False)
    print(f"💾 Saved cleaned file → {out_path}")

print("\n✅ Re-save complete. Check Drive → processed folder.")

💾 Saved cleaned file → /content/drive/MyDrive/ML_Projects/ieee-fraud-detection/processed/train_transaction_cleaned.csv
💾 Saved cleaned file → /content/drive/MyDrive/ML_Projects/ieee-fraud-detection/processed/train_identity_cleaned.csv
💾 Saved cleaned file → /content/drive/MyDrive/ML_Projects/ieee-fraud-detection/processed/test_transaction_cleaned.csv
💾 Saved cleaned file → /content/drive/MyDrive/ML_Projects/ieee-fraud-detection/processed/test_identity_cleaned.csv

✅ Re-save complete. Check Drive → processed folder.


In [16]:
for p in PROCESSED_DIR.glob("*.csv"):
    print(p.name, "done", round(p.stat().st_size / (1024*1024), 2), "MB")


train_transaction_cleaned.csv done 605.88 MB
train_identity_cleaned.csv done 23.79 MB
test_transaction_cleaned.csv done 546.33 MB
test_identity_cleaned.csv done 23.12 MB


Imputation Strategy

## 🧩 1.1.4 Imputation Strategy

**Objective:**  
Handle the remaining missing values in each dataset to ensure data completeness and consistency before integration and transformation.

---

### 🎯 Goals
After dropping highly sparse columns (>80% missing), we now fill the remaining missing values using a balanced approach that preserves the statistical integrity of each feature.

---

### ⚙️ Imputation Logic

#### 1️⃣ Numerical Features  
- Replace missing values with the **median** of each column.  
- Median is preferred over mean as it’s robust against outliers (common in transaction amounts).

#### 2️⃣ Categorical Features  
- Replace missing values with the **mode** (most frequent value).  
- If a column is entirely missing or has low diversity, fill with a constant placeholder `"missing"`.

#### 3️⃣ Datetime Features  
- Skip imputation for now — we’ll handle them later during feature transformation if needed.  

---

### 🧾 Expected Outputs
- Four new imputed files in the `processed/` directory:  
  - `train_transaction_imputed.csv`  
  - `train_identity_imputed.csv`  
  - `test_transaction_imputed.csv`  
  - `test_identity_imputed.csv`
- A report file summarizing total imputed values per column →  
  `artifacts/missing_reports/imputation_summary.json`

---

> 🧠 **Note:** Imputation happens **after dropping sparse columns** and before encoding or scaling steps in later phases.

In [17]:
import numpy as np
import pandas as pd

def split_cols_by_type(df: pd.DataFrame):
    num_cols  = df.select_dtypes(include=[np.number]).columns.tolist()
    cat_cols  = df.select_dtypes(include=["object", "category", "bool"]).columns.tolist()
    dt_cols   = df.select_dtypes(include=["datetime64[ns]", "datetime64[ns, UTC]"]).columns.tolist()
    other_cols = sorted(list(set(df.columns) - set(num_cols) - set(cat_cols) - set(dt_cols)))
    return num_cols, cat_cols, dt_cols, other_cols

for name, df in cleaned_datasets.items():
    num_cols, cat_cols, dt_cols, other_cols = split_cols_by_type(df)

    print(f"\n Dataset: {name}")
    print(f"  • numeric: {len(num_cols)} | categorical: {len(cat_cols)} | datetime: {len(dt_cols)} | other: {len(other_cols)}")

    # Sum of missing values
    total_missing = int(df.isna().sum().sum())
    print(f"  • total NaNs: {total_missing:,}")

    # Top 5 Nans
    if num_cols:
        num_miss = df[num_cols].isna().sum().sort_values(ascending=False).head(5)
        print("  • top numeric NaNs:")
        display(num_miss.to_frame("n_missing").T)

    if cat_cols:
        cat_miss = df[cat_cols].isna().sum().sort_values(ascending=False).head(5)
        print("  • top categorical NaNs:")
        display(cat_miss.to_frame("n_missing").T)

    if dt_cols:
        dt_miss = df[dt_cols].isna().sum().sort_values(ascending=False).head(5)
        print("  • top datetime NaNs:")
        display(dt_miss.to_frame("n_missing").T)


 Dataset: train_transaction
  • numeric: 325 | categorical: 14 | datetime: 0 | other: 0
  • total NaNs: 67,434,214
  • top numeric NaNs:


,V278,V277,V276,V275,V274
n_missing,460110,460110,460110,460110,460110


  • top categorical NaNs:


,R_emaildomain,M5,M7,M8,M9
n_missing,453249,350482,346265,346252,346252



 Dataset: train_identity
  • numeric: 17 | categorical: 15 | datetime: 0 | other: 0
  • total NaNs: 852,028
  • top numeric NaNs:


...

,R_emaildomain,M5,M4,M7,M8
n_missing,370821,309632,237745,235018,235004



 Dataset: test_identity
  • numeric: 17 | categorical: 15 | datetime: 0 | other: 0
  • total NaNs: 873,411
  • top numeric NaNs:


,id-18,id-03,id-04,id-32,id-14
n_missing,91032,75426,75426,71236,70550


  • top categorical NaNs:


,id-30,id-33,id-34,DeviceInfo,id-16
n_missing,71248,71236,69732,26850,16160


Numerical Imputation (median)

In [18]:
from tqdm import tqdm

imputation_summary = {}

for name, df in cleaned_datasets.items():
    num_cols, cat_cols, dt_cols, _ = split_cols_by_type(df)
    df_imputed = df.copy()
    summary = {}

    for col in tqdm(num_cols, desc=f"Imputing numeric cols in {name}"):
        n_missing = df_imputed[col].isna().sum()
        if n_missing > 0:
            median_val = df_imputed[col].median()
            df_imputed[col].fillna(median_val, inplace=True)
            summary[col] = int(n_missing)

    imputation_summary[name] = {"numeric_filled": summary}
    cleaned_datasets[name] = df_imputed  # overwrite with imputed version

print(" - Numeric imputation done. Missing values filled with column medians.")

Imputing numeric cols in train_transaction:   0%|          | 0/325 [00:00<?, ?it/s]/tmp/ipython-input-1474682562.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_imputed[col].fillna(median_val, inplace=True)
Imputing numeric cols in train_identity:   0%|          | 0/17 [00:00<?, ?it/s]/tmp/ipython-input-1474682562.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never 

 - Numeric imputation done. Missing values filled with column medians.


In [19]:
for name, df in cleaned_datasets.items():
    num_cols, cat_cols, dt_cols, _ = split_cols_by_type(df)
    n_missing_after = df[num_cols].isna().sum().sum()
    n_cat_missing   = df[cat_cols].isna().sum().sum()
    print(f"\n {name}")
    print(f"   • numeric NaNs remaining: {n_missing_after}")
    print(f"   • categorical NaNs remaining: {n_cat_missing}")


 train_transaction
   • numeric NaNs remaining: 0
   • categorical NaNs remaining: 3204208

 train_identity
   • numeric NaNs remaining: 0
   • categorical NaNs remaining: 274624

 test_transaction
   • numeric NaNs remaining: 0
   • categorical NaNs remaining: 2387365

 test_identity
   • numeric NaNs remaining: 0
   • categorical NaNs remaining: 300392


In [20]:
for name, df in cleaned_datasets.items():
    num_cols, cat_cols, dt_cols, _ = split_cols_by_type(df)
    df_imputed = df.copy()
    cat_summary = {}

    for col in cat_cols:
        n_missing = df_imputed[col].isna().sum()
        if n_missing > 0:
            if df_imputed[col].dropna().nunique() > 0:
                mode_val = df_imputed[col].mode()[0]
                df_imputed[col].fillna(mode_val, inplace=True)
            else:
                df_imputed[col].fillna("missing", inplace=True)
            cat_summary[col] = int(n_missing)

    imputation_summary[name]["categorical_filled"] = cat_summary
    cleaned_datasets[name] = df_imputed

print(" Categorical imputation done (mode or 'missing').")

/tmp/ipython-input-2872003630.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_imputed[col].fillna(mode_val, inplace=True)
/tmp/ipython-input-2872003630.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try us

 Categorical imputation done (mode or 'missing').


In [21]:
for name, df in cleaned_datasets.items():
    total_missing = int(df.isna().sum().sum())
    print(f"{name:<20} → total remaining NaNs: {total_missing}")

train_transaction    → total remaining NaNs: 0
train_identity       → total remaining NaNs: 0
test_transaction     → total remaining NaNs: 0
test_identity        → total remaining NaNs: 0


In [22]:
import json

# Save imputed versions
for name, df in cleaned_datasets.items():
    out_path = PROCESSED_DIR / f"{name}_imputed.csv"
    df.to_csv(out_path, index=False)
    print(f" Saved imputed file → {out_path}")

# Save imputation summary JSON
imputation_json_path = ARTIFACT_DIR / "imputation_summary.json"
with open(imputation_json_path, "w") as f:
    json.dump(imputation_summary, f, indent=2)

print(f"\n Imputation summary saved to: {imputation_json_path}")
print("All cleaned & imputed datasets saved successfully.")

 Saved imputed file → /content/drive/MyDrive/ML_Projects/ieee-fraud-detection/processed/train_transaction_imputed.csv
 Saved imputed file → /content/drive/MyDrive/ML_Projects/ieee-fraud-detection/processed/train_identity_imputed.csv
 Saved imputed file → /content/drive/MyDrive/ML_Projects/ieee-fraud-detection/processed/test_transaction_imputed.csv
 Saved imputed file → /content/drive/MyDrive/ML_Projects/ieee-fraud-detection/processed/test_identity_imputed.csv

 Imputation summary saved to: /content/drive/MyDrive/ML_Projects/ieee-fraud-detection/artifacts/missing_reports/imputation_summary.json
All cleaned & imputed datasets saved successfully.


## 🔗 1.2 Data Integration

**Objective:**  
Merge related tables (`transaction` and `identity`) to create unified datasets for both **training** and **testing**.  
This step ensures that all relevant features describing a transaction (behavioral, identity, device, etc.) are combined into a single table for modeling.

---

### 🎯 Why Integration?
The IEEE-CIS Fraud Detection dataset is split into two main parts:
1. **Transaction Table:** contains payment, card, and vendor details.
2. **Identity Table:** contains device, network, and user identity information.

Each table shares a common key column:
> `TransactionID`

We’ll use this key to merge the identity information into the corresponding transaction record.

---

### ⚙️ Integration Plan
1️⃣ Load imputed versions of all four datasets.  
2️⃣ Merge:
   - `train_transaction_imputed.csv` + `train_identity_imputed.csv` → **train_full**
   - `test_transaction_imputed.csv` + `test_identity_imputed.csv` → **test_full**

3️⃣ Verify:
   - Merge success (row counts, column counts, null checks)
   - No duplicate `TransactionID`s introduced

4️⃣ Save final integrated datasets:
   - `train_integrated.csv`
   - `test_integrated.csv`

---

> 🧠 **Note:** Integration must be done *after* cleaning and imputation to ensure consistent schemas between train and test datasets.


Load imputed datasets & verify merge keys

In [2]:
# Paths
imputed_paths = {
    "train_transaction": PROCESSED_DIR / "train_transaction_imputed.csv",
    "train_identity":    PROCESSED_DIR / "train_identity_imputed.csv",
    "test_transaction":  PROCESSED_DIR / "test_transaction_imputed.csv",
    "test_identity":     PROCESSED_DIR / "test_identity_imputed.csv",
}

# Load
imputed_data = {}
for name, path in imputed_paths.items():
    imputed_data[name] = pd.read_csv(path, low_memory=False)
    print(f" Loaded {name:<20} → shape: {imputed_data[name].shape}")

# Verify key existence
for name, df in imputed_data.items():
    has_key = "TransactionID" in df.columns
    n_unique = df["TransactionID"].nunique() if has_key else 0
    print(f"{name:<20} | has TransactionID: {has_key} | unique IDs: {n_unique:,}")

print("\n All datasets loaded and merge keys verified.")

 Loaded train_transaction    → shape: (590540, 339)
 Loaded train_identity       → shape: (144233, 32)
 Loaded test_transaction     → shape: (506691, 341)
 Loaded test_identity        → shape: (141907, 32)
train_transaction    | has TransactionID: True | unique IDs: 590,540
train_identity       | has TransactionID: True | unique IDs: 144,233
test_transaction     | has TransactionID: True | unique IDs: 506,691
test_identity        | has TransactionID: True | unique IDs: 141,907

 All datasets loaded and merge keys verified.


Merge train_transaction + train_identity

In [3]:
train_full = pd.merge(
    imputed_data["train_transaction"],
    imputed_data["train_identity"],
    on="TransactionID",
    how="left",
    suffixes=("", "_id")
)

print(" train_full created successfully.")
print(f"Shape before merge: transaction={imputed_data['train_transaction'].shape}, "
      f"identity={imputed_data['train_identity'].shape}")
print(f"Shape after merge : {train_full.shape}")

# Quick sanity checks
same_rows = len(train_full) == len(imputed_data["train_transaction"])
unique_ids = train_full["TransactionID"].is_unique
print(f"Row count preserved: {same_rows}")
print(f"Unique TransactionID: {unique_ids}")

 train_full created successfully.
Shape before merge: transaction=(590540, 339), identity=(144233, 32)
Shape after merge : (590540, 370)
Row count preserved: True
Unique TransactionID: True


Merge test_transaction + test_identity

In [4]:
test_full = pd.merge(
    imputed_data["test_transaction"],
    imputed_data["test_identity"],
    on="TransactionID",
    how="left",
    suffixes=("", "_id")
)

print(" test_full created successfully.")
print(f"Shape before merge: transaction={imputed_data['test_transaction'].shape}, "
      f"identity={imputed_data['test_identity'].shape}")
print(f"Shape after merge : {test_full.shape}")

# Sanity checks
same_rows = len(test_full) == len(imputed_data["test_transaction"])
unique_ids = test_full["TransactionID"].is_unique
print(f"Row count preserved: {same_rows}")
print(f"Unique TransactionID: {unique_ids}")

 test_full created successfully.
Shape before merge: transaction=(506691, 341), identity=(141907, 32)
Shape after merge : (506691, 372)
Row count preserved: True
Unique TransactionID: True


In [5]:
# Save to processed directory
train_path = PROCESSED_DIR / "train_integrated.csv"
test_path  = PROCESSED_DIR / "test_integrated.csv"

train_full.to_csv(train_path, index=False)
test_full.to_csv(test_path, index=False)

print(f" train_integrated.csv saved → {train_path}")
print(f" test_integrated.csv  saved → {test_path}")

# Quick summary
summary = pd.DataFrame({
    "dataset": ["train_full", "test_full"],
    "rows": [len(train_full), len(test_full)],
    "cols": [train_full.shape[1], test_full.shape[1]],
    "total_missing": [
        train_full.isna().sum().sum(),
        test_full.isna().sum().sum()
    ]
})

print("\n Integration summary:")
display(summary)

 train_integrated.csv saved → /content/drive/MyDrive/ML_Projects/ieee-fraud-detection/processed/train_integrated.csv
 test_integrated.csv  saved → /content/drive/MyDrive/ML_Projects/ieee-fraud-detection/processed/test_integrated.csv

 Integration summary:


,dataset,rows,cols,total_missing
0,train_full,590540,370,13835517
1,test_full,506691,372,11308304


## ⚙️ 1.3 Data Transformation

**Objective:**  
Transform the integrated datasets into a fully model-ready format by handling merge-level missingness, encoding categorical features, and scaling numerical features.

---

### 🎯 Why Transformation?
After merging `transaction` and `identity` tables, new missing values appear for transactions that had no matching identity record.  
These are *expected structural gaps*, not dirty data.  
To prepare for modeling, we must now ensure:
- All columns have consistent, machine-readable formats
- No `NaN`s remain in the integrated data
- Numerical features are scaled properly
- Categorical features are encoded for model input

---

### ⚙️ Transformation Plan

#### 1️⃣ Handle Merge-Level Missingness
- Fill numeric columns with **0** or median where appropriate  
- Fill categorical columns with `"missing"`  
- Create a binary **`has_identity`** flag (1 if identity info exists, 0 otherwise)

#### 2️⃣ Encoding
- Apply **Label Encoding** to low-cardinality categorical features  
- Apply **Frequency Encoding** (or Target Encoding later) to high-cardinality ones

#### 3️⃣ Scaling
- Normalize continuous features using `StandardScaler` or `RobustScaler`  
- Save the scaler object for consistent inference-time transformation

#### 4️⃣ Output
- `train_transformed.csv`  
- `test_transformed.csv`  
- `artifacts/transform_report.json` documenting all encodings & scalers used

---

> 🧠 **Note:** This phase ensures the data is clean, standardized, and fully numerical — ready for model training in Phase 4.


Handle merge-level missingness & add has_identity flag

In [3]:
import pandas as pd


# Reload integrated datasets
train_full = pd.read_csv(PROCESSED_DIR / "train_integrated.csv", low_memory=False)
test_full  = pd.read_csv(PROCESSED_DIR / "test_integrated.csv", low_memory=False)

print(f"train_full shape: {train_full.shape}")
print(f"test_full  shape: {test_full.shape}")

# ---- Add has_identity flag ----
# Transactions that have identity info will have non-null values in at least one id_* column
def add_identity_flag(df):
    id_cols = [c for c in df.columns if c.startswith("id_")]
    df["has_identity"] = df[id_cols].notna().any(axis=1).astype(int)
    return df

train_full = add_identity_flag(train_full)
test_full  = add_identity_flag(test_full)

# ---- Quick missing-value overview ----
def missing_overview(df, name):
    total_missing = df.isna().sum().sum()
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    categorical_cols = df.select_dtypes(exclude=[np.number]).columns
    print(f"\n{name}")
    print(f"   total missing values: {total_missing:,}")
    print(f"   numeric cols: {len(numeric_cols)} | categorical cols: {len(categorical_cols)}")
    print(f"   has_identity 1s: {df['has_identity'].sum():,} | 0s: {(df['has_identity']==0).sum():,}")

missing_overview(train_full, "train_full")
missing_overview(test_full, "test_full")

train_full shape: (590540, 370)
test_full  shape: (506691, 372)

train_full
   total missing values: 13,835,517
   numeric cols: 342 | categorical cols: 29
   has_identity 1s: 144,233 | 0s: 446,307

test_full
   total missing values: 11,308,304
   numeric cols: 344 | categorical cols: 29
   has_identity 1s: 0 | 0s: 506,691


In [4]:
def add_identity_flag_fixed(df):
    id_cols = [c for c in df.columns if c.startswith("id_")]
    if not id_cols:
        df["has_identity"] = 0
    else:
        df["has_identity"] = df[id_cols].notna().any(axis=1).astype(int)
    return df

train_full["has_identity"] = add_identity_flag_fixed(train_full)["has_identity"]
test_full["has_identity"]  = add_identity_flag_fixed(test_full)["has_identity"]

print(" Fixed has_identity assignment.")

missing_overview(train_full, "train_full (after fix)")
missing_overview(test_full, "test_full (after fix)")

 Fixed has_identity assignment.

train_full (after fix)
   total missing values: 13,835,517
   numeric cols: 342 | categorical cols: 29
   has_identity 1s: 144,233 | 0s: 446,307

test_full (after fix)
   total missing values: 11,308,304
   numeric cols: 344 | categorical cols: 29
   has_identity 1s: 0 | 0s: 506,691


As we can absorve here has_identity in the test lacks alot of feature value to make the thing more as a real world realistic most of attacks lacks some features identity such as : device fingerprint , IP patterns , browser IDs why are we exceeding further ignoring this step becuase we don't need to rely on that thing

Fill merge level NaNs (numeric → 0, categorical → "missing")

In [5]:
def fill_merge_level_nans(df, name):
    num_cols = df.select_dtypes(include=[np.number]).columns
    cat_cols = df.select_dtypes(exclude=[np.number]).columns

    df_filled = df.copy()
    df_filled[num_cols] = df_filled[num_cols].fillna(0)
    df_filled[cat_cols] = df_filled[cat_cols].fillna("missing")

    total_missing_after = int(df_filled.isna().sum().sum())
    print(f"{name:<12} → filled merge-level NaNs. Remaining missing: {total_missing_after}")
    return df_filled

train_full_filled = fill_merge_level_nans(train_full, "train_full")
test_full_filled  = fill_merge_level_nans(test_full, "test_full")

train_full   → filled merge-level NaNs. Remaining missing: 0
test_full    → filled merge-level NaNs. Remaining missing: 0


Encode categorical features (Label + Frequency)

In [4]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
import json

encoding_maps = {}

def encode_categoricals(df, name):
    df_encoded = df.copy()
    cat_cols = df_encoded.select_dtypes(exclude=[np.number]).columns

    print(f"\n{name}: encoding {len(cat_cols)} categorical columns...")

    for col in cat_cols:
        unique_count = df_encoded[col].nunique(dropna=False)

        # --- Low cardinality → Label Encoding ---
        if unique_count <= 20:
            le = LabelEncoder()
            df_encoded[col] = le.fit_transform(df_encoded[col])
            encoding_maps[col] = {
                "type": "label",
                "mapping": {str(k): int(v) for k, v in zip(le.classes_, le.transform(le.classes_))}
            }

        # --- High cardinality → Frequency Encoding ---
        else:
            freq = df_encoded[col].value_counts(dropna=False)
            df_encoded[col] = df_encoded[col].map(freq)
            encoding_maps[col] = {
                "type": "frequency",
                "mapping": {str(k): int(v) for k, v in freq.to_dict().items()}
            }

    print(f"{name} encoded successfully.")
    return df_encoded


# Encode datasets
train_encoded = encode_categoricals(train_full_filled, "train_full")
test_encoded  = encode_categoricals(test_full_filled, "test_full")

# Save encoding maps (safe JSON)
encoding_path = ARTIFACT_DIR / "encoding_maps.json"
with open(encoding_path, "w") as f:
    json.dump(encoding_maps, f, indent=2)

print(f"\n✅ Encoding maps saved successfully to {encoding_path}")

NameError: name 'train_full_filled' is not defined

In [3]:
train_encoded.to_csv(PROCESSED_DIR / "train_encoded.csv", index=False)
test_encoded.to_csv(PROCESSED_DIR / "test_encoded.csv", index=False)

print(" Encoded datasets saved to processed folder.")


NameError: name 'train_encoded' is not defined

In [2]:
train_encoded = pd.read_csv(PROCESSED_DIR / "train_encoded_partial.csv", low_memory=False)
test_encoded  = pd.read_csv(PROCESSED_DIR / "test_encoded_partial.csv",  low_memory=False)

print(f"train_encoded shape: {train_encoded.shape}")
print(f"test_encoded  shape: {test_encoded.shape}")

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/ML_Projects/ieee-fraud-detection/processed/train_encoded_partial.csv'